In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('movie.csv') # Veri setinin okunması.

In [3]:
df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [4]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [5]:
from nltk.corpus import stopwords
from textblob import Word

sw=stopwords.words("english") 

In [6]:
def data_cleaning(df,choice):
    df = df.applymap(lambda s: s.lower() if type(s) == str else s) 
    df["text"]=df["text"].str.replace("[^\w\s]","") 
    df["text"]=df["text"].str.replace("\d","") 
    df["text"]=df["text"].apply(lambda x: " ".join(x for x in x.split() if x not in sw)) 
    if choice == 1:
        df["text"]=df["text"].apply(lambda x: " ".join(Word(i).lemmatize() for i in x.split())) 
    elif choice == 2:
        df["text"]=df["text"].apply(lambda x: " ".join(Word(i).stem() for i in x.split())) 
    return df['text']

In [7]:
df['text']=data_cleaning(df, 1)

C:\Users\OUZHAN~1\AppData\Local\Temp/ipykernel_8724/1107926885.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"]=df["text"].str.replace("[^\w\s]","") # noktalama işaretlerinin silinmesi
C:\Users\OUZHAN~1\AppData\Local\Temp/ipykernel_8724/1107926885.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"]=df["text"].str.replace("\d","") # sayıların silinmesi


In [8]:
df.head()

,text,label
0,grew b watching loving thunderbird mate school...,0
1,put movie dvd player sat coke chip expectation...,0
2,people know particular time past like feel nee...,0
3,even though great interest biblical movie bore...,0
4,im die hard dad army fan nothing ever change g...,1


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
vectorizer = TfidfVectorizer(lowercase=False,ngram_range=(1,2),dtype=np.byte, max_features=10000) 
X = vectorizer.fit_transform(df['text'].tolist()) 

C:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:2029: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int8'> 'dtype' will be converted to np.float64.
  warnings.warn(


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,df['label'],random_state=1,test_size=0.25)

In [13]:
X_train = X_train.toarray()
X_test = X_test.toarray()

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [15]:
model = Sequential() # ANN modelinin tanımlanması.
model.add(Dense(activation = "relu", input_dim = X_train.shape[1], units = 8, kernel_initializer = 'uniform'))
model.add(Dense(activation = "sigmoid", units = 1, kernel_initializer = 'uniform'))
model.compile(optimizer = 'adam' , loss = 'mean_squared_error', metrics = ['accuracy'])

In [16]:
model.fit(X_train , y_train, epochs = 10, verbose=0, batch_size = 32) 
y_pred=model.predict(X_test, verbose=0).flatten().round()
print("Accuracy Score on traning data: ", model.evaluate(X_train,y_train, verbose=0)[1])
print("Accuracy Score on test data: ", model.evaluate(X_test,y_test, verbose=0)[1])
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Accuracy Score: ", accuracy_score(y_test,y_pred))

Accuracy Score on traning data:  0.990066647529602
Accuracy Score on test data:  0.8741000294685364
[[4296  695]
 [ 564 4445]]
              precision    recall  f1-score   support

           0       0.88      0.86      0.87      4991
           1       0.86      0.89      0.88      5009

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000

Accuracy Score:  0.8741


In [17]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression() 
model_lr=lr.fit(X_train,y_train) 
prediction_lr=model_lr.predict(X_test) 
print("Accuracy Score on traning data: ",lr.score(X_train,y_train))
print("Accuracy Score on test data: ",lr.score(X_test,y_test)) 
print(confusion_matrix(y_test, prediction_lr)) 
print(classification_report(y_test, prediction_lr)) 
print("Accuracy Score: ",accuracy_score(y_test, prediction_lr)) 

Accuracy Score on traning data:  0.9255666666666666
Accuracy Score on test data:  0.8913
[[4376  615]
 [ 472 4537]]
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      4991
           1       0.88      0.91      0.89      5009

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

Accuracy Score:  0.8913
